In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType
spark = SparkSession.builder.appName("Ngram_filter").master("spark://192.168.1.76:7077").getOrCreate()
sc = spark.sparkContext

In [2]:
df = spark.read.csv(['../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-0.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-1.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-2.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-3.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-4.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-5.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-6.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-7.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-8.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-9.csv'], sep="\t", header=False)

In [3]:
df = df.withColumnRenamed('_c0', 'ngram')
df = df.withColumnRenamed('_c1', 'year')
df = df.withColumnRenamed('_c2', 'match_count')
df = df.withColumnRenamed('_c3', 'page_count')
df = df.withColumnRenamed('_c4', 'volume_count')

In [4]:
df.count()

188660459

In [5]:
def is_alpha(x):
    return x.isalpha()

is_alpha_udf = udf(is_alpha, BooleanType())

In [6]:
df = df.filter(is_alpha_udf(df['ngram']))

In [7]:
df = df.sort(['ngram', 'year'], ascending=True).cache()

In [8]:
df.count()

169185297

In [9]:
df.coalesce(1).write.csv("eng-gb-all-only-letters")